In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import random
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_train = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_train.npy')
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_train.npy')
X_test = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_test.npy')
y_test = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_test.npy')
X_validation = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/X_validation.npy')
y_validation = np.load('/content/drive/MyDrive/Colab Notebooks/CS671/Assignment_2/y_validation.npy')

In [ ]:
X_train_2_4_6_7_8 = [X_train[0], X_train[2277], X_train[4554], X_train[6831], X_train[9108]]
X_validation_2_4_6_7_8 = [X_validation[0], X_validation[759], X_validation[1518], X_validation[2277], X_validation[3036]]

In [ ]:
plt.subplot(2,3,1),plt.imshow(X_train[0].reshape(28,28))
plt.subplot(2,3,2),plt.imshow(X_train[2277].reshape(28,28))
plt.subplot(2,3,3),plt.imshow(X_train[4554].reshape(28,28))
plt.subplot(2,3,4),plt.imshow(X_train[6831].reshape(28,28))
plt.subplot(2,3,5),plt.imshow(X_train[9108].reshape(28,28))

In [ ]:
dummy = list(zip(X_train, y_train))
random.shuffle(dummy)
X_train, y_train = zip(*dummy)
X_train = np.array(X_train)
y_train = np.array(y_train)

dummy = list(zip(X_test, y_test))
random.shuffle(dummy)
X_test, y_test = zip(*dummy)
X_test = np.array(X_test)
y_test = np.array(y_test)

dummy = list(zip(X_validation, y_validation))
random.shuffle(dummy)
X_validation, y_validation = zip(*dummy)
X_validation = np.array(X_validation)
y_validation = np.array(y_validation)

In [ ]:
X_train1=X_train/255
X_validation1=X_validation/255
X_test1=X_test/255

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.GaussianNoise(stddev=0.5, seed=55, input_shape=(784,)),
    tf.keras.layers.Dense(32, activation='sigmoid', name='bottle_neck'),
    tf.keras.layers.Dense(784, activation='sigmoid',name='output')
])

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=50,
    verbose=0,
    mode="min",
    restore_best_weights=True
)
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model.compile(optimizer='adam',loss='mse')

history = model.fit(X_train1, X_train1, epochs=500, batch_size=32, verbose=2,
          validation_data = (X_validation1, X_validation1),
          callbacks=[callback, model_checkpoint_callback])

model.load_weights(checkpoint_filepath)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig("model_loss_encoder.png")
plt.show()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import Sequential
denoise_model = Sequential()
denoise_model.add(model.get_layer('bottle_neck'))
denoise_model.add(model.get_layer('output'))

In [ ]:
pred = model.predict(np.array(X_train_2_4_6_7_8))
figure(figsize=(12,9),dpi=80)
plt.subplot(2,5,1),plt.imshow(X_train_2_4_6_7_8[0].reshape(28,28))
plt.subplot(2,5,2),plt.imshow(X_train_2_4_6_7_8[1].reshape(28,28))
plt.subplot(2,5,3),plt.imshow(X_train_2_4_6_7_8[2].reshape(28,28))
plt.subplot(2,5,4),plt.imshow(X_train_2_4_6_7_8[3].reshape(28,28))
plt.subplot(2,5,5),plt.imshow(X_train_2_4_6_7_8[4].reshape(28,28))
plt.subplot(2,5,6),plt.imshow(pred[0].reshape(28,28))
plt.subplot(2,5,7),plt.imshow(pred[1].reshape(28,28))
plt.subplot(2,5,8),plt.imshow(pred[2].reshape(28,28))
plt.subplot(2,5,9),plt.imshow(pred[3].reshape(28,28))
plt.subplot(2,5,10),plt.imshow(pred[4].reshape(28,28))
plt.suptitle('Reconstructed images from training dataset')
plt.savefig("Reconstructed_images_from_training_dataset")

In [ ]:
pred = model.predict(np.array(X_validation_2_4_6_7_8))
figure(figsize=(12,9),dpi=80)
plt.subplot(2,5,1),plt.imshow(X_validation_2_4_6_7_8[0].reshape(28,28))
plt.subplot(2,5,2),plt.imshow(X_validation_2_4_6_7_8[1].reshape(28,28))
plt.subplot(2,5,3),plt.imshow(X_validation_2_4_6_7_8[2].reshape(28,28))
plt.subplot(2,5,4),plt.imshow(X_validation_2_4_6_7_8[3].reshape(28,28))
plt.subplot(2,5,5),plt.imshow(X_validation_2_4_6_7_8[4].reshape(28,28))
plt.subplot(2,5,6),plt.imshow(pred[0].reshape(28,28))
plt.subplot(2,5,7),plt.imshow(pred[1].reshape(28,28))
plt.subplot(2,5,8),plt.imshow(pred[2].reshape(28,28))
plt.subplot(2,5,9),plt.imshow(pred[3].reshape(28,28))
plt.subplot(2,5,10),plt.imshow(pred[4].reshape(28,28))
plt.suptitle('Reconstructed images from validation dataset')
plt.savefig("Reconstructed_images_from_validation_dataset")

In [ ]:
# Average reconstruction error
pred = model.predict(X_train1)
dummy = []
for i in range(len(pred)):
    dummy.append(pred[i]- X_train1[i])
dummy = np.array(dummy)
dummy1 = dummy.flatten()
for i in range(len(dummy1)):
    dummy1[i] = dummy1[i]*dummy1[i]
reconstruction_error_train= np.sum(dummy1)/len(dummy1)
print("Final Average reconstruction error for training data is:", reconstruction_error_train)

pred = model.predict(X_validation1)
dummy = []
for i in range(len(pred)):
    dummy.append(pred[i]- X_validation1[i])
dummy = np.array(dummy)
dummy1 = dummy.flatten()
for i in range(len(dummy1)):
    dummy1[i] = dummy1[i]*dummy1[i]
reconstruction_error_validation= np.sum(dummy1)/len(dummy1)
print("Final Average reconstruction error for validation data is:", reconstruction_error_validation)

pred = model.predict(X_test1)
dummy = []
for i in range(len(pred)):
    dummy.append(pred[i]- X_test1[i])
dummy = np.array(dummy)
dummy1 = dummy.flatten()
for i in range(len(dummy1)):
    dummy1[i] = dummy1[i]*dummy1[i]
reconstruction_error_test= np.sum(dummy1)/len(dummy1)
print("Final Average reconstruction error for testing data is:", reconstruction_error_test)

In [ ]:
# Weight visualization
z=np.array(model.layers[1].trainable_weights)
W=np.array(z[0])
W = W.T
from matplotlib.pyplot import figure
figure(figsize=(16,12), dpi=80)
for i in range(len(W)):
  plt.subplot(8,4,i+1)
  plt.imshow(W[i].reshape(28,28))
plt.suptitle('Weight visualization (plot of weights from the input layer to the compressed layer)')
plt.savefig("weight_visualization.png")

In [ ]:
encoder = tf.keras.models.Sequential(model.get_layer('bottle_neck'))

In [ ]:
X_train_encoder=encoder.predict(X_train1)
X_validation_encoder=encoder.predict(X_validation1)

In [ ]:
Classifier_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='sigmoid', name='hidden_layer_1', input_shape=(32,)),
    tf.keras.layers.Dense(16, activation='sigmoid', name='hidden_layer_2'),
    tf.keras.layers.Dense(8, activation='sigmoid', name='hidden_layer_3'),
    tf.keras.layers.Dense(5, activation='softmax')
])

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=50,
    verbose=0,
    mode="min",
    restore_best_weights=True
)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

Classifier_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08),
    loss=tf.keras.losses.CategoricalCrossentropy(),  
    metrics = ['accuracy'] 
)

history = Classifier_model.fit(X_train_encoder, y_train, epochs=5000, batch_size=len(X_train_encoder), verbose=2,   
          validation_data = (X_validation_encoder, y_validation),
          callbacks=[callback, model_checkpoint_callback]
)

Classifier_model.load_weights(checkpoint_filepath)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.savefig("model_accuracy.png")
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.savefig("model_loss_decoder.png")
plt.show()

In [ ]:
Classifier_model.summary()

In [ ]:
X_test_encoder=encoder(X_test1)
pred = Classifier_model.predict(X_test_encoder)
y_pred_test = []
for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred_test.append(dummy1)

y_test_1 = []
for dummy in y_test:
    dummy1 = (np.argmax(dummy, axis=0))
    y_test_1.append(dummy1)

y_pred_test = np.array(y_pred_test)

cf_matrix_test = confusion_matrix(y_test_1, y_pred_test)
print(cf_matrix_test)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_test_plot = sns.heatmap(cf_matrix_test, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')   # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_test_plot.get_figure()
fig.savefig("confusion_matrix_test.png") 

In [ ]:
X_train_encoder=encoder(X_train1)
pred = Classifier_model.predict(X_train_encoder)
y_pred_train = []

for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred_train.append(dummy1)

y_train_1 = []
for dummy in y_train:
    dummy1 = (np.argmax(dummy, axis=0))
    y_train_1.append(dummy1)

y_pred_train = np.array(y_pred_train)

cf_matrix_train = confusion_matrix(y_train_1, y_pred_train)
print(cf_matrix_train)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_train_plot = sns.heatmap(cf_matrix_train, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')    # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_train_plot.get_figure()
fig.savefig("confusion_matrix_train.png")

In [ ]:
X_validation_encoder=encoder(X_validation1)
pred = Classifier_model.predict(X_validation_encoder)
y_pred_validation = []
for dummy in pred:
    dummy1 = (np.argmax(dummy, axis=0))
    y_pred_validation.append(dummy1)

y_validation_1 = []
for dummy in y_validation:
    dummy1 = (np.argmax(dummy, axis=0))
    y_validation_1.append(dummy1)

y_pred_validation = np.array(y_pred_validation)

cf_matrix_validation = confusion_matrix(y_validation_1, y_pred_validation)
print(cf_matrix_validation)

x_axis_labels = [2,4,6,7,8]
y_axis_labels = [2,4,6,7,8]
cf_matrix_validation_plot = sns.heatmap(cf_matrix_validation, xticklabels=x_axis_labels, yticklabels=y_axis_labels, annot=True, fmt='g')   # fmt='g': general format, annot:If True, write the data value in each cell.
fig = cf_matrix_validation_plot.get_figure()
fig.savefig("confusion_matrix_validation.png") 

In [ ]:
loss, acc = Classifier_model.evaluate(X_test_encoder, y_test, verbose=2)
print('Test Accuracy: %.3f' % acc)